In [54]:
import requests
from IPython.display import HTML
import json
import pandas as pd
import numpy as np
import re
import time
import pathlib

In [55]:
subscription_key = 'e41c34b3bcf841d0859d6aba9e8e1494'
assert subscription_key
text_analytics_base_url = "https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/"

language_api_url = text_analytics_base_url + "languages"
print(language_api_url)

sentiment_api_url = text_analytics_base_url + "sentiment"
print(sentiment_api_url)

https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/languages
https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/sentiment


In [56]:
#function to return tweet language and sentiment score

def language_and_sentiment(df):
    
    #Language
    id_count = 0
    documents = {'documents': []}
    
    #compile language document
    for text in df['clean_text']:        
        documents['documents'].append({'id': '{}'.format(id_count), 'text': text})
        id_count += 1
        
    #feed compiled doc to API    
    headers   = {"Ocp-Apim-Subscription-Key": subscription_key}
    response  = requests.post(language_api_url, headers=headers, json=documents)
    languages = response.json()
    
    new_lang_dict = {}
    for entry in languages['documents']:
        new_lang_dict[entry['id']] = entry['detectedLanguages'][0]['iso6391Name']
    
    
    langs = []
    for i in range(0,len(df)):
        if str(i) in new_lang_dict.keys():
            langs.append(new_lang_dict[str(i)])
        else:
            langs.append(np.nan)
        
    

    #append languages to df
    df['language'] = langs
    
    
    ## Sentiment
    id_count = 0
    documents = {'documents': []}
    
    #compile sentiment document
    for text in df['clean_text']:
        documents['documents'].append({'id': '{}'.format(id_count), 'language': langs[id_count], 'text': text})
        id_count += 1
        
    #feed compiled sentiment doc to API
    headers   = {"Ocp-Apim-Subscription-Key": subscription_key}
    response  = requests.post(sentiment_api_url, headers=headers, json=documents)
    sentiments = response.json()
    
    new_sent_dict = {}
    for entry in sentiments['documents']:
        new_sent_dict[entry['id']] = entry['score']
    
    
    sents = []
   
    for i in range(0,len(df)):
        if str(i) in new_sent_dict.keys():
            sents.append(new_sent_dict[str(i)])
        else:
            sents.append(np.nan)
        
        
    
    df['sentiment'] = sents
    
    return df


In [57]:
#function to clean tweet text

def clean_tweet(row):
    text = re.sub('Retweeted.*\):','',row['text'])
    text = re.sub('(http.*$)','',text)
    text = re.sub('(pic.twitter.*$)','',text)
    text = re.sub('([@#][\w_-]+)','',text)
    text = re.sub('([@] [^ ]+)','', text)
    text = re.sub('([#] [^ ]+)','', text)
    text = re.sub('[^\w]', ' ', text)
    text = re.sub('(['  '.*])', ' ', text)
    return text

In [58]:
def chunker(seq, size):
    return (seq[pos:pos + size].copy() for pos in range(0, len(seq), size))

In [60]:
#run through csvs in the directory, process and save in new folder

open_directory = pathlib.Path('data/clean')
save_path = 'data/sentiment'

for csv in open_directory.glob('*.csv'):
         
    all_tweets = []
    for tweets in pd.read_csv(csv, sep=';', chunksize = 10000,iterator=True, low_memory=False, error_bad_lines = False):

    #tweets = pd.read_csv(csv, sep=';', error_bad_lines = False)
    
    
        #binary indicator of whether the tweet was a retweet
        tweets['retweet'] = tweets['text'].apply(lambda x: re.sub(r'Retweeted.*\):','',x))
    
        #clean tweet text
        tweets['clean_text'] = tweets.apply(clean_tweet, axis=1)
        tweets = tweets[tweets['clean_text'].str.contains('([a-zA-Z]+)', regex=True)]
    
        chunk_dfs = []
    
        for i in chunker(tweets,500):
            time.sleep(10)
            chunk_dfs.append(language_and_sentiment(i))
    
        tweets_with_sent = pd.concat(chunk_dfs)
        all_tweets.append(tweets_with_sent)
    
    
    final = pd.concat(all_tweets)
    save_file = (save_path + str(csv)[10:-7] + 'sent.csv')
    
    final.to_csv(save_file)


https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/languages
https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/sentiment


/home/alex/anaconda3/envs/hackathon/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/languages
https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/sentiment
https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/languages
https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/sentiment
https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/languages
https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/sentiment
https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/languages
https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/sentiment


b'Skipping line 4073: expected 10 fields, saw 12\n'
b'Skipping line 14044: expected 10 fields, saw 11\n'
b'Skipping line 30941: expected 10 fields, saw 11\nSkipping line 33717: expected 10 fields, saw 12\n'


https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/languages
https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/sentiment
https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/languages
https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/sentiment


b'Skipping line 17709: expected 10 fields, saw 11\nSkipping line 18724: expected 10 fields, saw 11\n'
b'Skipping line 24832: expected 10 fields, saw 11\n'


https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/languages
https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/sentiment


b'Skipping line 14699: expected 10 fields, saw 12\nSkipping line 15499: expected 10 fields, saw 12\n'
b'Skipping line 28221: expected 10 fields, saw 11\n'


https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/languages
https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/sentiment
https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/languages
https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/sentiment


b'Skipping line 6353: expected 10 fields, saw 11\n'
